## Priority Queue

Creates a priority queue of time series to be analysed based on their probability of being an astrophysical source.

#### Import packages and read in pre-processed dataframes/arrays

In [30]:
import sklearn.ensemble
import sklearn.multiclass
import sklearn.model_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

unlabelledDf = pd.read_pickle('./processed.files/raw/unlabelledDf')
preProcFeats = np.load('./processed.files/labelled/features.npy')
preProcLabels = np.load('./processed.files/labelled/labels.npy')
rawfeats = np.load('./processed.files/raw/features.npy')

#### Define the k-fold shuffling method and use the RandomForest method to fit the OneVsRest classifier on all of the data

In [31]:
randForest = sklearn.multiclass.OneVsRestClassifier(sklearn.ensemble.RandomForestClassifier()) #classifier for multiple labels

clf = sklearn.multiclass.OneVsRestClassifier(estimator=sklearn.ensemble.RandomForestClassifier(bootstrap=True, 
            class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=-1,
            oob_score=False, random_state=420, verbose=0,
            warm_start=False),
            n_jobs=1)

fitmodel = clf.fit(preProcFeats, preProcLabels)

#### Predict probabilities to be in class 9, add them to the dataframe and sort by descending probability

In [33]:
pred = fitmodel.predict_proba(rawfeats)
pred9 = pred[:,7]

print pred9.shape
print unlabelledDf.shape

unlabelledDf['P(class9)'] = pred9

print unlabelledDf.shape
print unlabelledDf['P(class9)']

sortedDf = unlabelledDf.sort_values('P(class9)', axis =1,  ascending = False)

(58212,)
(58212, 411)
(58212, 411)
0        0.053333
1        0.037975
2        0.044944
3        0.053333
4        0.097222
5        0.074627
6        0.050633
7        0.050000
8        0.050633
9        0.044944
10       0.054054
11       0.049383
12       0.063291
13       0.055556
14       0.050633
15       0.051282
16       0.054054
17       0.054054
18       0.067568
19       0.071429
20       0.057971
21       0.064935
22       0.089552
23       0.070588
24       0.052632
25       0.054054
26       0.051948
27       0.052632
28       0.057143
29       0.048780
           ...   
73252    0.011905
73253    0.011905
73254    0.012048
73255    0.011628
73256    0.011905
73257    0.012048
73258    0.012048
73259    0.011765
73260    0.011905
73261    0.011905
73262    0.023810
73263    0.000000
73264    0.022222
73265    0.022222
73266    0.021739
73267    0.000000
73268    0.000000
73269    0.022222
73270    0.000000
73271    0.019231
73272    0.012821
73273    0.021739
73274    0.

KeyError: 'P(class9)'